In [1]:
import ee
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
import folium
import time

ee.Initialize()

In [2]:
def p2r(point, area_size = 100, field_size = 44):
    dist_lat = abs(area_size / 110.540)/1000
    dist_lon = abs(area_size / (111.320 * np.cos(point[0])))/1000
    area = {"type": "FeatureCollection", "features": [{"type": "Feature", "properties": {},"geometry": {"type": "Polygon", "coordinates": [[[point[0] - dist_lon/2, point[1] - dist_lat/2], [point[0] + dist_lon/2, point[1] - dist_lat/2], [point[0] + dist_lon/2, point[1] + dist_lat/2], [point[0] - dist_lon/2, point[1] + dist_lat/2], [point[0] - dist_lon/2, point[1] - dist_lat/2]]]}}]}
    dist_lat = abs(field_size / 110.540)/1000
    dist_lon = abs(field_size / (111.320 * np.cos(point[0])))/1000
    soccer={"type": "FeatureCollection", "features": [{"type": "Feature", "properties": {}, "geometry": {"type": "Polygon", "coordinates": [[[point[0] - dist_lon/2, point[1] - dist_lat/2], [point[0] + dist_lon/2, point[1] - dist_lat/2], [point[0] + dist_lon/2, point[1] + dist_lat/2], [point[0] - dist_lon/2,point[1] + dist_lat/2], [point[0] - dist_lon/2, point[1] - dist_lat/2]]]}}]}
    coords_area = area['features'][0]['geometry']['coordinates']
    aoi_area = ee.Geometry.Polygon(coords_area)
    coords_soccer = soccer['features'][0]['geometry']['coordinates']
    aoi_soccer = ee.Geometry.Polygon(coords_soccer)
    return(aoi_area, aoi_soccer)

aoi, aoi_sub = p2r([23.33873748779297, 40.67725325396418], area_size = 100, field_size = 44)

In [3]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds Earth Engine layers to a folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name = name,
        overlay = True,
        control = True).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [4]:
# im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
#            .filterBounds(aoi)
#            .filterDate(ee.Date('2019-09-01'),ee.Date('2020-01-31'))
# #            .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
# #            .filter(ee.Filter.eq('relativeOrbitNumber_start', 154))
#            .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
#            .sort('date'))

# timestamplist = (im_coll.aggregate_array('date')
#                  .map(lambda d: ee.String('T').cat(ee.String(d)))
#                  .getInfo())

# def clip_img(img):
#     """Clips a list of images."""
#     return ee.Image(img).clip(aoi)

# im_list = im_coll.toList(im_coll.size())
# im_list = ee.List(im_list.map(clip_img))

# im_list.length().getInfo()

# def selectvv(current):
#     return ee.Image(current).select('VV')

# vv_list = im_list.map(selectvv)

# location = aoi.centroid().coordinates().getInfo()[::-1]
# mp = folium.Map(location=location, zoom_start=10)
# rgb_images = (ee.Image.rgb(vv_list.get(150), vv_list.get(151), vv_list.get(152))
#               .log10().multiply(10))
# mp.add_ee_layer(rgb_images, {'min': -20,'max': 0}, 'rgb composite')
# mp.add_child(folium.LayerControl())
# # display(mp)

In [5]:
aoi, aoi_sub = p2r([22.7480, 40.6750], area_size = 3000, field_size = 200)

im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
           .filterBounds(aoi)
           .filterDate(ee.Date('2018-06-06'),ee.Date('2018-06-18'))
#            .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
#            .filter(ee.Filter.eq('relativeOrbitNumber_start', 154))
           .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
           .sort('date'))

acq_times = im_coll.aggregate_array('system:time_start').getInfo()

im_list = im_coll.toList(im_coll.size())
im1 = ee.Image(im_list.get(0)).select('VV').clip(aoi)
im2 = ee.Image(im_list.get(4)).select('VV').clip(aoi)
# [time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]

# Decision threshold alpha/2:
ratio = im1.divide(im2)
m = 3.5
dt = f.ppf(0.0005, 2*m, 2*m)

# LRT statistics.
q1 = im1.divide(im2)
q2 = im2.divide(im1)

# Change map with 0 = no change, 1 = decrease, 2 = increase in intensity.
c_map = im1.multiply(0).where(q2.lt(dt), 1)
c_map = c_map.where(q1.lt(dt), 2)

# Mask no-change pixels.
c_map = c_map.updateMask(c_map.gt(0))

# Display map with red for increase and blue for decrease in intensity.
location = aoi.centroid().coordinates().getInfo()[::-1]
mp = folium.Map(
    location=location,
    zoom_start=14)
folium.TileLayer('OpenStreetMap').add_to(mp)
mp.add_ee_layer(ratio,
                {'min': 0, 'max': 20, 'palette': ['black', 'white']}, 'Ratio')
mp.add_ee_layer(c_map,
                {'min': 0, 'max': 2, 'palette': ['black', 'blue', 'red']},
                'Change Map')
mp.add_child(folium.LayerControl())
folium.LatLngPopup().add_to(mp)
display(mp)

In [6]:
[time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]
mp.save("not_flooded.html")

In [7]:
aoi, aoi_sub = p2r([22.7337, 40.6732], area_size = 5000, field_size = 200)

im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
           .filterBounds(aoi)
           .filterDate(ee.Date('2018-05-06'),ee.Date('2018-05-14'))
#            .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
#            .filter(ee.Filter.eq('relativeOrbitNumber_start', 154))
           .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
           .sort('date'))

acq_times = im_coll.aggregate_array('system:time_start').getInfo()

im_list = im_coll.toList(im_coll.size())
im1 = ee.Image(im_list.get(0)).select('VV').clip(aoi)
im2 = ee.Image(im_list.get(4)).select('VV').clip(aoi)
# [time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]

# Decision threshold alpha/2:
ratio = im1.divide(im2)
m = 3.5
dt = f.ppf(0.0005, 2*m, 2*m)

# LRT statistics.
q1 = im1.divide(im2)
q2 = im2.divide(im1)

# Change map with 0 = no change, 1 = decrease, 2 = increase in intensity.
c_map = im1.multiply(0).where(q2.lt(dt), 1)
c_map = c_map.where(q1.lt(dt), 2)

# Mask no-change pixels.
c_map = c_map.updateMask(c_map.gt(0))

# Display map with red for increase and blue for decrease in intensity.
location = aoi.centroid().coordinates().getInfo()[::-1]
mp = folium.Map(
    location=location,
    zoom_start=14)
folium.TileLayer('OpenStreetMap').add_to(mp)
mp.add_ee_layer(ratio,
                {'min': 0, 'max': 20, 'palette': ['black', 'white']}, 'Ratio')
mp.add_ee_layer(c_map,
                {'min': 0, 'max': 2, 'palette': ['black', 'blue', 'red']},
                'Change Map')
mp.add_child(folium.LayerControl())
folium.LatLngPopup().add_to(mp)
display(mp)

In [8]:
[time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]
mp.save("flooded.html")